# NOTE: I have tried to calculate the answer for both questions but identified a bug that I am still trying to understand. My code does not run because of syntax and indexing problems when calculating the posterior. (Details commented in the code.)

In [1]:
from scipy import stats
import pandas as pd 
import numpy as np

data = pd.read_csv("socialmobility.csv")
print(data)
print("Number of data points:", len(data))

          father           son  count
0           farm          farm    703
1           farm     unskilled   1478
2           farm       skilled   1430
3           farm  professional   1109
4      unskilled          farm     58
5      unskilled     unskilled   1756
6      unskilled       skilled   1630
7      unskilled  professional   1568
8        skilled          farm     63
9        skilled     unskilled   1453
10       skilled       skilled   2068
11       skilled  professional   2483
12  professional          farm     61
13  professional     unskilled    749
14  professional       skilled   1183
15  professional  professional   3315
Number of data points: 16


a. What is the posterior 95% probability interval over the probability that a son will
become a skilled laborer if his father was an unskilled laborer?

In [27]:
theta = data[data.father == 'unskilled']
print(theta)
print(type(theta))
where = pd.DataFrame(theta.loc[:, 'count'])
where

      father           son  count
4  unskilled          farm     58
5  unskilled     unskilled   1756
6  unskilled       skilled   1630
7  unskilled  professional   1568
<class 'pandas.core.frame.DataFrame'>


,count
4,58
5,1756
6,1630
7,1568


In [28]:
k = len(theta)
simplex = 1/k # probability vector over the categories
n = data.sum()

# For the prior on theta, we’ll assume a uniform distribution,
# by initializing the Dirichlet prior with a series of 1s for the parameter a, 
# one for each of the k possible outcomes. This is similar to 
# initializing a Beta distribution as Beta(1,1), which corresponds to 
# the uniform distribution

alpha_priors = [1]*k
print(alpha_priors)

# Wikipedia formula: posterior hyperparameters = alpha + count of how many x_i's
alpha_posteriors = []
for i in range(k):
    where = theta['count'] # subsettin the count columns...
    new = alpha_priors[i] + where[i] # NOTE: "where[i]" IS GIVING ME ERRORS...
    # Break down of problem:
        # 1. There is something wrong with my syntax for subsetting
        # 2. Even if this were right, I am indexing values that don't exist within the unskilled['count'] range
    # Possible other syntax that might fix the error:
    # theta['count'[i]] -> no
    # unkilled = data.loc[lamda data: data.father == "unskilled", "count"] and then subset? -> no
    
    alpha_posteriors.append(new)

print("Alpha posteriors:", alpha_posteriors)

# Since Dirichlet is the conjugate prior to a multinimial likelihood,
# the posterior distribution is also Dirichlet

distribution = stats.dirichlet(alpha_posteriors)
samples = distribution.rvs(size=1000)
print(samples)
print("The 95% CI for the probability that a son will become a skilled laborer \nif his father \
was an unskilled laborer is:", np.percentile(samples[:,2], [2.5, 97.5]))

[1, 1, 1, 1]


KeyError: 0

b. What is the posterior 95% probability interval over the probability that a father
works on a farm if his son works as a professional?

In [ ]:
theta_2 = data[data.son == 'professional']
alpha_priors_2 = np.repeat(1, k)

# Trying a different method here to extrac the data and getting the same problem :(
alpha_posteriors_2 = [alpha_priors_2[i] + theta_2['count'].iloc[i] for i in range(k)] 

distribution_2 = stats.dirichlet(alpha_posteriors)
samples_2 = distribution_2.rvs(size=1000)
print("The 95% CI for the probability that a father works on a farm \nif his son \
works as a professional is:", np.percentile(samples_2[:,0], [2.5, 97.5]))